## Import Dependencies

In [36]:
import pandas as pd
import numpy as np
from datetime import datetime

from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

import os
from dotenv import load_dotenv

In [37]:

load_dotenv()

SQLALCHEMY_DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(SQLALCHEMY_DATABASE_URL)

In [38]:
chapters_df = pd.read_sql("""
    SELECT
        c.id AS chapter_id,
        c.chapter_title AS chapter_name,
        c.course_id,
        c.owner_id AS user_id
    FROM chapters c
""", engine)
chapters_df

,chapter_id,chapter_name,course_id,user_id
0,1,Chapter 1: Introduction to Statistics,1,1
1,2,Types of Data,1,1
2,3,Measures of Central Tendency,1,1
3,4,Measures of Dispersion,1,1
4,5,Probability Basics,1,1
5,6,Conditional Probability,1,1
6,7,Random Variables,1,1
7,8,Probability Distributions,1,1
8,9,Sampling Techniques,1,1
9,10,Statistical Applications,1,1


In [47]:
time_df = pd.read_sql("""
    SELECT
        chapter_id,
        owner_id AS user_id,

        SUM(CASE WHEN activity_type = 'view_content' THEN duration_seconds ELSE 0 END) AS view_content,
        SUM(CASE WHEN activity_type = 'summary' THEN duration_seconds ELSE 0 END) AS time_summary,
        SUM(CASE WHEN activity_type = 'ask_question' THEN duration_seconds ELSE 0 END) AS time_ask,
        SUM(CASE WHEN activity_type = 'mcq' THEN duration_seconds ELSE 0 END) AS time_mcq,

        MAX(session_end) AS last_activity
    FROM learning_sessions
    WHERE is_valid = True
    GROUP BY chapter_id, owner_id
""", engine)
time_df

,chapter_id,user_id,view_content,time_summary,time_ask,time_mcq,last_activity
0,28,1,2325,0,0,0,2026-01-05 10:09:47+00:00
1,27,1,0,1872,0,2264,2026-01-05 10:02:06+00:00
2,50,1,2358,0,0,0,2026-01-09 17:41:46+00:00
3,21,1,0,2353,0,1978,2026-01-05 09:43:54+00:00
4,31,1,0,2429,0,2220,2026-01-05 15:35:20+00:00
5,48,1,0,2258,0,2238,2026-01-09 18:17:21+00:00
6,1,1,0,2124,0,2454,2026-01-07 17:35:59+00:00
7,32,1,2331,0,0,0,2026-01-05 15:41:10+00:00
8,51,1,0,2509,0,2294,2026-01-05 10:24:59+00:00
9,9,1,0,2203,0,2516,2026-01-07 18:18:21+00:00


In [48]:
mcq_df = pd.read_sql("""
    SELECT
        chapter_id,
        owner_id AS user_id,
        COUNT(*) AS mcq_attempts,
        AVG(score_percentage) AS mcq_avg_score,
        MAX(attempted_at) AS last_mcq_attempt
    FROM mcq_attempts
    GROUP BY chapter_id, owner_id
""", engine)

mcq_df


,chapter_id,user_id,mcq_attempts,mcq_avg_score,last_mcq_attempt
0,28,1,5,92.0,2026-01-05 18:30:00+00:00
1,27,1,5,72.0,2026-01-05 19:00:00+00:00
2,50,1,5,84.0,2026-01-10 03:05:00+00:00
3,21,1,5,72.0,2026-01-05 16:00:00+00:00
4,31,1,5,76.0,2026-01-05 22:40:00+00:00
5,48,1,5,80.0,2026-01-10 01:12:00+00:00
6,32,1,5,84.0,2026-01-05 20:59:00+00:00
7,1,1,5,88.0,2026-01-07 22:40:00+00:00
8,51,1,5,80.0,2026-01-05 21:00:00+00:00
9,9,1,5,68.0,2026-01-08 01:12:00+00:00


In [49]:
df = chapters_df \
    .merge(time_df, on=["chapter_id", "user_id"], how="left") \
    .merge(mcq_df, on=["chapter_id", "user_id"], how="left")


In [50]:
df.fillna({
    "view_content":0,
    "time_summary": 0,
    "time_ask": 0,
    "time_mcq": 0,
    "mcq_attempts": 0,
    "mcq_avg_score": 0
}, inplace=True)


In [53]:
df["total_time"] = df["time_summary"] + df["time_ask"] + df["time_mcq"] + df["view_content"]

df["score_efficiency"] = df["mcq_avg_score"] / (df["total_time"] + 1)

now = pd.Timestamp.utcnow()
df["last_activity"] = pd.to_datetime(df["last_activity"])
df["inactive_days"] = (now - df["last_activity"]).dt.days
df["inactive_days"].fillna(999, inplace=True)


/var/folders/j2/bny1m67s2_d0jg2q7zt0v2nh0000gn/T/ipykernel_5691/1548114636.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["inactive_days"].fillna(999, inplace=True)


In [54]:
def label_learning_state(row):
    if row["score_efficiency"] < 0.02 and row["inactive_days"] > 7:
        return "revise_urgent"
    elif row["score_efficiency"] < 0.03:
        return "practice_more"
    elif row["score_efficiency"] > 0.06 and row["inactive_days"] < 3:
        return "mastered"
    else:
        return "on_track"

df["learning_state"] = df.apply(label_learning_state, axis=1)

In [55]:
# Create final dataframe with selected columns
# Use .copy() to avoid SettingWithCopyWarning when modifying later
final_df = df[[
    "user_id",
    "course_id",
    "chapter_id",
    "chapter_name",
    "view_content",
    "time_summary",
    "time_ask",
    "time_mcq",
    "total_time",
    "mcq_attempts",
    "mcq_avg_score",
    "score_efficiency",
    "learning_state"
]].copy()

final_df


,user_id,course_id,chapter_id,chapter_name,view_content,time_summary,time_ask,time_mcq,total_time,mcq_attempts,mcq_avg_score,score_efficiency,learning_state
0,1,1,1,Chapter 1: Introduction to Statistics,0,2124,0,2454,4578,5,88.0,0.019218,practice_more
1,1,1,2,Types of Data,2343,0,0,0,2343,5,96.0,0.040956,on_track
2,1,1,3,Measures of Central Tendency,0,2288,0,2125,4413,5,80.0,0.018124,practice_more
3,1,1,4,Measures of Dispersion,2288,0,0,0,2288,5,76.0,0.033202,on_track
4,1,1,5,Probability Basics,0,2417,0,2218,4635,5,72.0,0.015531,practice_more
5,1,1,6,Conditional Probability,2450,0,0,0,2450,5,92.0,0.037536,on_track
6,1,1,7,Random Variables,0,2186,0,1972,4158,5,68.0,0.016350,practice_more
7,1,1,8,Probability Distributions,2104,0,0,0,2104,5,92.0,0.043705,on_track
8,1,1,9,Sampling Techniques,0,2203,0,2516,4719,5,68.0,0.014407,practice_more
9,1,1,10,Statistical Applications,2024,0,0,0,2024,5,72.0,0.035556,on_track


In [57]:
X = df[
    [
        "time_summary",
        "time_ask",
        "time_mcq",
        "mcq_attempts",
        "inactive_days"
    ]
]

In [58]:
y = df["learning_state"]

In [59]:
X

,time_summary,time_ask,time_mcq,mcq_attempts,inactive_days
0,2124,0,2454,5,-1
1,0,0,0,5,-1
2,2288,0,2125,5,-1
3,0,0,0,5,-1
4,2417,0,2218,5,-1
5,0,0,0,5,-1
6,2186,0,1972,5,-1
7,0,0,0,5,-1
8,2203,0,2516,5,-1
9,0,0,0,5,-1


In [60]:
y

0     practice_more
1          on_track
2     practice_more
3          on_track
4     practice_more
5          on_track
6     practice_more
7          on_track
8     practice_more
9          on_track
10    practice_more
11         on_track
12    practice_more
13         on_track
14    practice_more
15         on_track
16    practice_more
17         on_track
18    practice_more
19         on_track
20    practice_more
21         on_track
22    practice_more
23         on_track
24    practice_more
25         on_track
26    practice_more
27         on_track
28    practice_more
29         on_track
30    practice_more
31         on_track
32    practice_more
33         on_track
34    practice_more
35         on_track
36    practice_more
37         on_track
38    practice_more
39         on_track
40    practice_more
41         on_track
42    practice_more
43         on_track
44    practice_more
45         on_track
46    practice_more
47         on_track
48    practice_more
49    practice_more


In [61]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42
)

In [63]:
model = DecisionTreeClassifier(
    max_depth=3,
    random_state=42
)

model.fit(X_train, y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [64]:
accuracy = model.score(X_test, y_test)
accuracy

1.0

In [65]:
df["predicted_state"] = le.inverse_transform(
    model.predict(X)
)

In [66]:
priority_map = {
    "revise_urgent": 0,
    "practice_more": 1,
    "on_track": 2,
    "mastered": 3
}

df["priority"] = df["predicted_state"].map(priority_map)

recommendations = df.sort_values(
    by=["priority", "inactive_days"],
    ascending=[True, False]
)


In [67]:
recommendations[
    [
        "course_id",
        "chapter_id",
        "chapter_name",
        "predicted_state",
        "score_efficiency",
        "inactive_days"
    ]
]

,course_id,chapter_id,chapter_name,predicted_state,score_efficiency,inactive_days
20,4,21,Introduction to DSA,practice_more,0.016620,1
22,4,23,Arrays,practice_more,0.022310,1
24,4,27,Queues,practice_more,0.017404,1
26,4,29,Graphs,practice_more,0.020446,1
28,5,31,Introduction to DBMS,practice_more,0.016344,1
30,5,33,Relational Algebra,practice_more,0.016242,1
32,5,35,Advanced SQL,practice_more,0.013099,1
34,5,37,Functional Dependencies,practice_more,0.015551,1
36,5,39,Indexing,practice_more,0.016579,1
48,4,51,Linked Lists,practice_more,0.016653,1
